# Inconsistent data entry
## 1. Tutoriel

In [ ]:
!pip install fuzzywuzzy

In [1]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# read in all our data
professors = pd.read_csv("../Data/data1/pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

## Effectuez un pré-traitement de texte

Nous allons commencer par jeter un rapide coup d'œil aux premières lignes de données.

In [2]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


Supposons que nous souhaitions nettoyer la colonne "Pays" pour nous assurer qu'il n'y a pas d'incohérences dans la saisie des données. Nous pourrions bien sûr parcourir et vérifier chaque ligne à la main, et corriger manuellement les incohérences lorsque nous les trouvons. Il existe cependant un moyen plus efficace de le faire !

In [3]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

Rien qu'en regardant cela, je peux voir quelques problèmes dus à une saisie de données incohérente : ' Germany', et 'germany', par exemple, ou ' New Zealand' et 'New Zealand'.

La première chose que je vais faire est de tout mettre en minuscules (je peux le changer à la fin si je veux) et de supprimer tous les espaces blancs au début et à la fin des cellules. Les incohérences dans les majuscules et les espaces blancs de fin sont très courantes dans les données de texte et vous pouvez corriger un bon 80 % de vos incohérences de saisie de données de texte en procédant ainsi.

In [4]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

Ensuite, nous allons nous attaquer à des incohérences plus difficiles.

## Utiliser la correspondance approximative pour corriger la saisie de données incohérente

D'accord, jetons un autre coup d'œil à la colonne "Country" et voyons s'il y a plus de nettoyage de données que nous devons faire.

In [5]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

On dirait qu'il y a une autre incohérence :  'southkorea' et 'south korea' devraient être identiques.

Nous allons utiliser le package [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) pour aider à identifier les chaînes les plus proches les unes des autres. Cet ensemble de données est suffisamment petit pour que nous puissions probablement corriger les erreurs à la main, mais cette approche ne s'adapte pas bien. (Voudriez-vous corriger un millier d'erreurs à la main ? Et dix mille ? Automatiser les choses le plus tôt possible est généralement une bonne idée. En plus, c'est amusant !)

> **Correspondance approximative :** Le processus de recherche automatique de chaînes de texte très similaires à la chaîne cible. En général, une chaîne est considérée comme "plus proche" d'une autre d'autant moins que vous auriez besoin de changer de caractères si vous transformiez une chaîne en une autre. Ainsi, "apple" et "snapple" sont à deux changements l'un de l'autre (ajoutez "s" et "n") tandis que "in" et "on" et un changement (remplacez "i" par "o"). Vous ne pourrez pas toujours vous fier à 100% à la correspondance approximative, mais cela finira généralement par vous faire gagner au moins un peu de temps.

Fuzzywuzzy renvoie un rapport donné à deux chaînes. Plus le rapport est proche de 100, plus la distance d'édition entre les deux chaînes est petite. Ici, nous allons obtenir les dix chaînes de notre liste de villes qui ont la distance la plus proche de "d.i khan".

In [6]:
# get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('austria', 33),
 ('ireland', 33),
 ('pakistan', 32),
 ('portugal', 32),
 ('scotland', 32),
 ('australia', 30)]

Nous pouvons voir que deux des éléments dans les villes sont très proches de "south korea": "south korea" et "southkorea". Remplaçons toutes les lignes de notre colonne "Country" qui ont un ratio > 47 avec "south korea". 

Pour ce faire, je vais écrire une fonction. (C'est une bonne idée d'écrire une fonction à usage général que vous pouvez réutiliser si vous pensez avoir à faire une tâche spécifique plus d'une ou deux fois. Cela vous évite d'avoir à copier et coller du code trop souvent, ce qui permet de gagner du temps et peut aider éviter les erreurs.)

In [7]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

Maintenant que nous avons une fonction, nous pouvons la tester !

In [8]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done!


Et maintenant, vérifions à nouveau les valeurs uniques dans notre colonne "Country" et assurons-nous que nous avons bien rangé la "south korea".

In [9]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

## 2. Exercice

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# read in all our data
professors = pd.read_csv("../Data/data1/pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

Ensuite, nous allons refaire tout le travail que nous avons fait dans le tutoriel.

In [ ]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

# get the top 10 closest matches to "south korea"
countries = professors['Country'].unique()
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")
    
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")
countries = professors['Country'].unique()

1) Examiner une autre colonne

Écrivez le code ci-dessous pour jeter un œil à toutes les valeurs uniques dans la colonne "Graduated from".

In [ ]:
# TODO:

Avez-vous remarqué des incohérences dans les données ? L'une des incohérences dans les données peut-elle être corrigée en supprimant les espaces blancs au début et à la fin des cellules ?

Une fois que vous avez répondu à ces questions, exécutez la cellule de code ci-dessous pour obtenir un crédit pour votre travail.

In [ ]:
# TODO:

2) Faire un pré-traitement de texte

Convertissez chaque entrée de la colonne "Graduated from" dans le DataFrame des professeurs pour supprimer les espaces blancs au début et à la fin des cellules.

In [ ]:
# TODO:

3) Continuer à travailler avec les pays

Dans le tutoriel, nous nous sommes concentrés sur le nettoyage des incohérences dans la colonne "Country". Exécutez la cellule de code ci-dessous pour afficher la liste des valeurs uniques avec lesquelles nous avons terminé.

In [ ]:
# get all the unique values in the 'City' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

Examinez à nouveau la colonne "Pays" et voyez s'il y a plus de nettoyage de données que nous devons faire.

Il semble que « usa » et « usofa » devraient être le même pays. Corrigez la colonne « Country » dans le cadre de données afin que « usofa » apparaisse à la place de « usa ».

In [ ]:
# TODO: